In [1]:
import pandas as pd
import config_prod as config
#import json
from datetime import date, timedelta
import phood_api
import weeklymain

In [2]:
raw = pd.DataFrame(weeklymain.masterlist) 

In [3]:
rawfile=raw

In [4]:
raw.columns = ['Date_Produced', 'Log_ID', 'Item_Name', 'Item_Type', 'PLU', 'Logged_Time', 'Meal', 'Action_Taken', 'Action_Reason', 'Quantity', 'Pan_ID', 'Pan_Weight', 'Unit', 'Station', 'Employee_Name','imageId', 'Location_ID','Location', 'Served','Saved','Sold','Shrink']

In [5]:
import pandas_profiling
#pandas_profiling.ProfileReport(raw)

## Remove Unit, Meal, Item Type, Employee Name columns

In [6]:
raw = raw.drop(columns=['Unit','Meal','Item_Type','Employee_Name'])

In [7]:
raw['Date_Produced'] = pd.to_datetime(raw['Date_Produced'])
raw['Logged_Time'] = pd.to_datetime(raw['Logged_Time'])

In [8]:
raw['Logged_Date'] = raw['Logged_Time'].dt.date
#raw['Logged_Time'] = raw['Logged_Time'].dt.time

In [9]:
raw['Actual_Weight'] = raw['Quantity'] - raw['Pan_Weight']

In [10]:
raw = raw.drop(columns=['Log_ID','Date_Produced','Logged_Time','Action_Taken','Action_Reason','Pan_ID','Pan_Weight','imageId'])

In [11]:
file = raw.groupby(['Location_ID','Station','Logged_Date','PLU','Item_Name','Location'],as_index=False).sum()
#file

In [12]:
file = file.reindex(columns = ['Logged_Date', 'Station', 'PLU', 'Item_Name', 'Served','Saved','Shrink','Sold','Quantity','Actual_Weight','Location_ID','Location'])


In [13]:
negativeweights = file[(file.Served < 0) | (file.Saved < 0) | (file.Shrink < 0)]

In [14]:
#negativeweights

In [15]:
file = file[(file['Served'] >= 0) & (file['Saved'] >= 0) & (file['Shrink'] >= 0) & (file['Sold'] >=0)].reset_index(drop=True) 
file = file.round(2)
file['PLU'] = pd.to_numeric(file.PLU)
#file


In [16]:
file0 = file.drop(columns=['Quantity','Actual_Weight'])
#file0

In [17]:
file0.columns = ['Date', 'Venue','PLU', 'Item Name', 'Served', 'Saved', 'Shrink', 'Sold', 'Location_ID', 'Store']
file0 = file0.reindex(columns = ['Date','Store','Venue','PLU', 'Item Name', 'Served', 'Saved', 'Shrink', 'Sold', 'Location_ID'])
#file0

In [18]:
#startdate = pd.to_datetime("2019-12-18").date()
#file0 = file0[file0['Date'] > startdate]
#file0

In [19]:
file1 = file0[file0['Location_ID']==51]
file2 = file0[file0['Location_ID']==52]
file3 = file0[file0['Location_ID']==53]
file4 = file0[file0['Location_ID']==54]
file5 = file0[file0['Location_ID']==55]
file0 = file0.drop(columns=['Location_ID'])
file1 = file1.drop(columns=['Location_ID'])
file2 = file2.drop(columns=['Location_ID'])
file3 = file3.drop(columns=['Location_ID'])
file4 = file4.drop(columns=['Location_ID'])
file5 = file5.drop(columns=['Location_ID'])

#file1

In [20]:
with pd.ExcelWriter('weeklyreport.xlsx') as writer:  # doctest: +SKIP
    file0.to_excel(writer, sheet_name='weeklyreport', index = False)
    workbook  = writer.book
    worksheet = writer.sheets['weeklyreport']
    worksheet.set_column(0, 3, 12)
    worksheet.set_column(4, 4, 35)
    file1.to_excel(writer, sheet_name='Dedham_weeklyreport', index = False)
    worksheet = writer.sheets['Dedham_weeklyreport']
    worksheet.set_column(0, 3, 12)
    worksheet.set_column(4, 4, 35)
    file2.to_excel(writer, sheet_name='Hingham_weeklyreport', index = False)
    worksheet = writer.sheets['Hingham_weeklyreport']
    worksheet.set_column(0, 3, 12)
    worksheet.set_column(4, 4, 35)
    file3.to_excel(writer, sheet_name='Westford_weeklyreport', index = False)
    worksheet = writer.sheets['Westford_weeklyreport']
    worksheet.set_column(0, 3, 12)
    worksheet.set_column(4, 4, 35)
    file4.to_excel(writer, sheet_name='WestHartford_weeklyreport', index = False)
    worksheet = writer.sheets['WestHartford_weeklyreport']
    worksheet.set_column(0, 3, 12)
    worksheet.set_column(4, 4, 35)
    file5.to_excel(writer, sheet_name='Cranston_weeklyreport', index = False)
    worksheet = writer.sheets['Cranston_weeklyreport']
    worksheet.set_column(0, 3, 12)
    worksheet.set_column(4, 4, 35)
    #rawfile.to_excel(writer, sheet_name='Raw Logs', index = False)
